In [1]:
import sys
sys.path.append('team_11_ml_ops_module')  # Reemplaza 'team_11' con la ruta correcta de tu carpeta

from load_data import load_data
from explore_data import explore_data
from modeling.train import train_logistic_regression, train_random_forest, train_with_pca

import mlflow
import mlflow.sklearn

from sklearn.model_selection import train_test_split
#from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
import itertools

# Se leen los datos de divorcio

In [2]:
datapath = 'DATA/divorce.csv'
data = load_data(datapath)

# Se genera un mlflow con regresion logistica

In [3]:
mlflow.set_tracking_uri("http://localhost:8080")
mlflow.set_experiment(f"/team11/prediccion_divorsio")

<Experiment: artifact_location='mlflow-artifacts:/126551647513598786', creation_time=1728861523722, experiment_id='126551647513598786', last_update_time=1728861523722, lifecycle_stage='active', name='/team11/prediccion_divorsio', tags={}>

In [4]:
with mlflow.start_run() as run:
    # Load the diabetes dataset.
    X = data.drop('Class', axis=1)
    y = data['Class']
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

    # Explorar datos
    explore_data(data)

    # Log data split information
    mlflow.log_param("train_data_size", len(X_train))
    mlflow.log_param("test_data_size", len(X_test))

    # 2. Create a Pipeline for Preprocessing and Model Training
    pipeline = Pipeline([
        ("scaler", StandardScaler()),
        ("model", LogisticRegression(C=0.1, max_iter=1000, penalty='l1', solver='saga'))
    ])

    # 3. Train the Model
    pipeline.fit(X_train, y_train)

    # 4. Evaluate the Model
    y_pred = pipeline.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)

    # Log model performance metrics
    mlflow.log_metric("accuracy", accuracy)

    # Generate and print classification report
    report = classification_report(y_test, y_pred)
    print(report)
    mlflow.log_text(report, "classification_report.txt")  # Save report to artifacts

    # Generate and print confusion matrix
    cm = confusion_matrix(y_test, y_pred)
    print(cm)
    mlflow.log_text(str(cm), "confusion_matrix.txt")  # Save matrix to artifacts

    # 5. Log Model with MLflow
    mlflow.sklearn.log_model(pipeline, "Divorce_Prediction_Model")

    # 6. Add any relevant tags or information
    mlflow.set_tag("developer", "Team 11")  # For example
    mlflow.set_tag("model_type", "LogisticRegression")

print("MLflow experiment completed. Check mlruns folder for results.")

     Atr1  Atr2  Atr3  Atr4  Atr5  Atr6  Atr7  Atr8  Atr9  Atr10  ...  Atr46  \
0       2     2     4     1     0     0     0     0     0      0  ...      2   
1       4     4     4     4     4     0     0     4     4      4  ...      2   
2       2     2     2     2     1     3     2     1     1      2  ...      3   
3       3     2     3     2     3     3     3     3     3      3  ...      2   
4       2     2     1     1     1     1     0     0     0      0  ...      2   
..    ...   ...   ...   ...   ...   ...   ...   ...   ...    ...  ...    ...   
165     0     0     0     0     0     0     0     0     0      0  ...      1   
166     0     0     0     0     0     0     0     0     0      0  ...      4   
167     1     1     0     0     0     0     0     0     0      1  ...      3   
168     0     0     0     0     0     0     0     0     0      0  ...      3   
169     0     0     0     0     0     0     0     1     0      0  ...      3   

     Atr47  Atr48  Atr49  Atr50  Atr51 

2024/10/20 23:45:07 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2024/10/20 23:45:07 INFO mlflow.tracking._tracking_service.client: 🏃 View run skittish-toad-935 at: http://localhost:8080/#/experiments/126551647513598786/runs/faf65a3b32ac47989db29e2c1312ccf8.
2024/10/20 23:45:07 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:8080/#/experiments/126551647513598786.


MLflow experiment completed. Check mlruns folder for results.


# Pipeline de todos los modelos

 - 1. Regresion logistica
   2. Random Forest
   3. PCA + Regresion logistica

In [9]:
# Define una función para el entrenamiento y la evaluación
def run_experiment(model_name, model_func, X, y, param_grid):
    print(param_grid)
    run_counter = 1
    for param_combination in itertools.product(*param_grid.values()):
        # Crear nombre único para la run
        run_name = f"{model_name}_Run_{run_counter}_Params{param_combination}"

        with mlflow.start_run(run_name=run_name):
            mlflow.log_param("model_name", model_name)
            mlflow.log_param("parameters", param_combination)

            # Entrenamiento del modelo
            model, X_test, y_test = model_func(X, y, param_combination[0], param_combination[1], param_combination[2], param_combination[3])
            mlflow.sklearn.log_model(model, "model")  # Registra el modelo en MLflow
    
            # Evaluación del modelo
            y_pred = model.predict(X_test)
    
            accuracy = accuracy_score(y_test, y_pred)
            precision = precision_score(y_test, y_pred)
            recall = recall_score(y_test, y_pred)
            f1 = f1_score(y_test, y_pred)
    
    
            mlflow.log_metric("accuracy", accuracy)
            mlflow.log_metric("precision", precision)
            mlflow.log_metric("recall", recall)
            mlflow.log_metric("f1_score", f1)
    
            print(f"Modelo: {model_name}")
            print(f"Accuracy: {accuracy}")
            print(f"Precision: {precision}")
            print(f"Recall: {recall}")
            print(f"F1-score: {f1}")
        # Incrementar el contador de runs
        run_counter += 1

# Preprocesamiento, entrenamiento y evaluación con MLflow

param_grid = {
  'C': [0.1, 1, 10, 10001, 0.1],
  'penalty': ['l1', 'l2'],
  'solver': ['liblinear', 'saga'],
  'max_iter':[1000]
}

# Regresión Logística
run_experiment("Logistic Regression", train_logistic_regression, X, y, param_grid)

# Random Forest
param_grid = {
  'n_estimators': [100],
  'max_depth': [None, 5, 10],
  'min_samples_split': [2, 5, 10],
  'min_samples_leaf': [1, 2, 4]
}
run_experiment("Random Forest", train_random_forest, X, y, param_grid)

# PCA + Regresión Logística
param_grid = {
  'C': [0.01, 0.1, 1, 10, 100],
  'penalty': ['l1', 'l2'],
  'solver': ['liblinear', 'saga'],
  'max_iter':[1000]
}
run_experiment("PCA + Logistic Regression", train_with_pca, X, y, param_grid)

{'C': [0.1, 1, 10, 10001, 0.1], 'penalty': ['l1', 'l2'], 'solver': ['liblinear', 'saga'], 'max_iter': [1000]}


2024/10/20 23:53:33 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2024/10/20 23:53:33 INFO mlflow.tracking._tracking_service.client: 🏃 View run Logistic Regression_Run_1_Params(0.1, 'l1', 'liblinear', 1000) at: http://localhost:8080/#/experiments/126551647513598786/runs/6f91b48a55d34fcd926c1a60df432166.
2024/10/20 23:53:33 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:8080/#/experiments/126551647513598786.


Modelo: Logistic Regression
Accuracy: 0.9705882352941176
Precision: 1.0
Recall: 0.95
F1-score: 0.9743589743589743


2024/10/20 23:53:37 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2024/10/20 23:53:37 INFO mlflow.tracking._tracking_service.client: 🏃 View run Logistic Regression_Run_2_Params(0.1, 'l1', 'saga', 1000) at: http://localhost:8080/#/experiments/126551647513598786/runs/f7ecc57abc75441a8835f76e6e2aa38b.
2024/10/20 23:53:37 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:8080/#/experiments/126551647513598786.


Modelo: Logistic Regression
Accuracy: 0.9705882352941176
Precision: 1.0
Recall: 0.95
F1-score: 0.9743589743589743


...

Modelo: PCA + Logistic Regression
Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1-score: 1.0


2024/10/20 23:58:01 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2024/10/20 23:58:01 INFO mlflow.tracking._tracking_service.client: 🏃 View run PCA + Logistic Regression_Run_19_Params(100, 'l2', 'liblinear', 1000) at: http://localhost:8080/#/experiments/126551647513598786/runs/0cafaeebbb22489087e5bbe562775e4c.
2024/10/20 23:58:01 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:8080/#/experiments/126551647513598786.


Modelo: PCA + Logistic Regression
Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1-score: 1.0


C:\Users\a.ockrassa.morales\Documents\Otto\CosasOtto\Python\mlops\Lib\site-packages\sklearn\linear_model\_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
2024/10/20 23:58:05 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2024/10/20 23:58:05 INFO mlflow.tracking._tracking_service.client: 🏃 View run PCA + Logistic Regression_Run_20_Params(100, 'l2', 'saga', 1000) at: http://localhost:8080/#/experiments/126551647513598786/runs/839454546cdd4d4aa0cb0796fff4dbd9.
2024/10/20 23:58:05 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:8080/#/experiments/126551647513598786.


Modelo: PCA + Logistic Regression
Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1-score: 1.0


In [8]:
import itertools
i = 0
for param_combination in itertools.product(*param_grid.values()):
    print(i, param_combination)
    i = i +1

0 (100, None, 2, 1)
1 (100, None, 2, 2)
2 (100, None, 2, 4)
3 (100, None, 5, 1)
4 (100, None, 5, 2)
5 (100, None, 5, 4)
6 (100, None, 10, 1)
7 (100, None, 10, 2)
8 (100, None, 10, 4)
9 (100, 5, 2, 1)
10 (100, 5, 2, 2)
11 (100, 5, 2, 4)
12 (100, 5, 5, 1)
13 (100, 5, 5, 2)
14 (100, 5, 5, 4)
15 (100, 5, 10, 1)
16 (100, 5, 10, 2)
17 (100, 5, 10, 4)
18 (100, 10, 2, 1)
19 (100, 10, 2, 2)
20 (100, 10, 2, 4)
21 (100, 10, 5, 1)
22 (100, 10, 5, 2)
23 (100, 10, 5, 4)
24 (100, 10, 10, 1)
25 (100, 10, 10, 2)
26 (100, 10, 10, 4)
